In [1]:
import cx_Oracle
import pandas as pd
import requests
from xml.etree.ElementTree import fromstring

In [2]:
access_key="bgOnt78reFNsTUJuAwlI30JDObTxX6hbJCxyApJCtuf3xjJZJ/mOs8Vhg3GZAsLc1fXTkQ9sjq0mTEupWDdyyA=="

In [3]:
con = cx_Oracle.connect('open_source/1111@localhost:1521/xe')
cur = con.cursor()

In [8]:
TOTAL_NUM = 102
MAX_TRANSACTION = 30
column_list = ['adtFrDd','sgguNm', 'sidoNm', 'spclAdmTyCd', 'telno','yadmNm']
column_list = {'adtFrDd':'운영가능일자','sgguNm':'시군구명', 'sidoNm':'시도명', 'spclAdmTyCd':'구분코드', 'telno': '전화번호','yadmNm':'기관명'}


In [5]:
def xml_to_oracle(data_root, data_root_name):
    for row in data_root.iter(data_root_name):
            sql_insert = '''
            insert into covid_hospital(기관명,구분코드,시도명,시군구명,운영가능일자) 
            values(:기관명,:구분코드,:시도명,:시군구명,:운영가능일자)
            '''
            기관명 = row.find('yadmNm').text
            구분코드 = row.find('spclAdmTyCd').text
            시도명 = row.find('sidoNm').text
            시군구명 = row.find('sgguNm').text
            운영가능일자 = row.find('adtFrDd').text
            
            cur.execute(sql_insert, (기관명, 구분코드, 시도명, 시군구명,운영가능일자))
            con.commit()
      

In [10]:
def get_request_url(page_no):
    url = "http://apis.data.go.kr/B551182/pubReliefHospService/getpubReliefHospList"
    params = {'serviceKey': access_key , 'numOfRows' : MAX_TRANSACTION, 'pageNo' : page_no}

    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.text
        # return response.content
    else:
        print(f"비정상 response: status_code => {response.status_code} ")
        return None

In [12]:
request_total_num = ( TOTAL_NUM // MAX_TRANSACTION) + 1
rename_dict = {'adtFrDd': '운영가능일자', 'sgguNm': '시군구명', 'sidoNm': '시도명', 'spclAdmTyCd': '구분코드', 'telno': '전화번호',
                   'yadmNm': '기관명'}
REDEFINE_COLUMNS = ['기관명','구분코드','시도명','시군구명','전화번호','운영가능일자']
all_data = pd.DataFrame(columns=REDEFINE_COLUMNS)
for page_no in range (1, request_total_num +1):
    raw_xml = get_request_url(page_no)
    root = fromstring(raw_xml)

    data_root = root.find('body').find('items')
    xml_to_oracle(data_root, 'item')